In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

import pandas as pd
import numpy as np
from copy import deepcopy

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time, auto_save_pickle

from models.fmin2 import fmin2, p2_lr, p2_lgbm
from models.nested_validation import *
from models.features import fixed_makedata, salvador_wrap, pretrained_wrap
from models.display import pickle2df

# models
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings("ignore")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


### loading the dataset

In [2]:
dic = load_pickle('../data/dic_20191203.pickle')
ls = list(dic.keys())

time: 216 ms


In [3]:
def inner_CV(corp, medel1, method, model2_name):
    ''' 
    speficify the parameters to tune
    '''
    if model2_name =='LR':
        spaces = {'model1': model1, 'model2': LogisticRegression, 'method': method,
                  'p2': p2_lgbm, 'corp': corp}
    elif model2_name =='LGBM':
        spaces = {'model1': model1, 'model2': LGBMClassifier, 'method': method,
          'p2': p2_lgbm, 'corp': corp}
    else:
        print('cannot detect this model2')
        return
    
    ''' how fmin2 works
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2

time: 25.4 ms


### Example of training the best model in our paper
word embedding (glove) and nutritional properties: ['wv','nu','scale']

In [ ]:
'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = pretrained_wrap(knowns, 'glove-wiki-gigaword-300')

for method in [['wv','nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method, 'LGBM')
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-12-03 15:48:44.590906
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 256, 'max_depth': 16, 'learning_rate': 0.15, 'gamma': 0.65, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 5, 'subsample': 0.7}
2019-12-03 15:48:44.780922
[0.8343558282208587, 0.8322981366459627, 0.802547770700637, 0.8674698795180723, 0.8917197452229298]
0.8456782720616921
-0.8456782720616921 -0.8456782720616921
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 512, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.55, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.75, 'bagging_fraction': 0.5, 'bagging_freq': 5, 'subsample': 0.7}
2019-12-03 15:49:59.298873
[0.8263473053892216, 0.8383233532934131, 0.7792207792207793, 0.8520710059171598, 0.8875]
0.8366924887641147
-0.8366924887641147 -0.8456782720616921
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 64, 'max_depth': 8, 'learning_

### Examples of training these models with logistic regression
Naive Bayes BoW: ['wc','nb'] <br>
Nutritions only: ['nu','scale'] <br>
Nutritions + Naive Bayes BoW: ['wc','nb','nu','scale']

In [ ]:
'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
model1 = None
state = 2022
p2 = p2_lr

for method in [['wc','nb'], ['nu','scale'], ['wc','nb','nu','scale']]:
    print('state', state)
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method, 'LR')
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2), method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))